# Full processing for moraine dam surface change
Quinn Brencher

In [1]:
# import packages
import os
import pyproj
pyproj.datadir.set_data_dir('/srv/conda/envs/mintpy/share/proj')
os.environ["PROJ_DATA"]="/srv/conda/envs/mintpy/share/proj"
from glob import glob
import geopandas as gpd
from pathlib import Path
from typing import List, Union
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
import rioxarray
import xarray as xr
import rasterio as rio
import datetime
from datetime import datetime, timedelta
import sys
from mintpy.utils import readfile, writefile, utils as ut, plot
from mintpy.cli import view, tsview, plot_network, plot_transection
from mintpy.view import prep_slice, plot_slice
import gc
import pandas as pd
import seaborn as sns
from scipy.stats import linregress
from rasterio.enums import Resampling
from matplotlib.colors import LogNorm

sns.set_theme()

In [2]:
# set up paths, directories, and variables for this specific dam
data_path = '/home/jovyan/repos/incubator_moraine_dam_analysis/data'
asc_burst = '012_023788_IW2'
des_burst = '048_101863_IW3'
asc_burst_path = f'{data_path}/data_igrams/{asc_burst}'
des_burst_path = f'{data_path}/data_igrams/{des_burst}'

dam_name = 'nagma'
proc_path = f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{dam_name}'

# define reference point, [y, x]
reference_point = [3073709.5, 495052.7]

# define crs 
crs = 32645

In [3]:
# read in dam polygon
dam_fn = f'../../mapping/polygons/{dam_name}_md.shp'
dam_gdf = gpd.read_file(dam_fn)
dam_gdf = dam_gdf.to_crs(crs) # reproject in case of mistake

# load in moving area polygon
moving_fn = f'../../mapping/polygons/{dam_name}_moving.shp'
moving_gdf = gpd.read_file(moving_fn)

# load in reference area polygon
reference_fn = f'../../mapping/polygons/{dam_name}_reference.shp'
reference_gdf = gpd.read_file(reference_fn)

# load in stable area polygon
stable_fn = f'../../mapping/polygons/{dam_name}_stable.shp'
stable_gdf = gpd.read_file(stable_fn).to_crs(crs)

# # load in water mask
# water_fn = f'../../mapping/polygons/water_mask.shp'
# water_gdf = gpd.read_file(water_fn).to_crs(crs)

# pad dam bounds by 5 km 
padding = 5000
aoi_extent = [dam_gdf.bounds.minx.item()-padding,
               dam_gdf.bounds.maxy.item()+padding,
               dam_gdf.bounds.maxx.item()+padding,
               dam_gdf.bounds.miny.item()-padding]

## crop InSAR data

In [4]:
# def clip_hyp3_products_to_common_overlap(data_path: Union[str, Path], overlap: List[float]) -> None:    
#     files_for_mintpy = ['_rng_off.tif', '_snr.tif', '_conncomp.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']
#     for extension in files_for_mintpy:
#         print(f'working on {extension}') 
#         for file in data_path.rglob(f'*{extension}'):
#             dst_file = file.parent / f'{file.stem}_{dam_name}{file.suffix}'
#             if not os.path.isfile(dst_file):
#                 gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [5]:
# # # clip igram products to dam aoi
# # clip_hyp3_products_to_common_overlap(Path(asc_burst_path), aoi_extent)
# clip_hyp3_products_to_common_overlap(Path(des_burst_path), aoi_extent)

# load coherence time series

In [6]:
# functions to load interferogram tifs to xarray
def parse_file(file_path):
    data = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(': ')
            data[key] = value
    return data

def xr_read_geotif(geotif_file_path, masked=True):
    """
    Reads in single or multi-band GeoTIFF as dask array.
    Inputs
    ----------
    GeoTIFF_file_path : GeoTIFF file path
    Returns
    -------
    ds : xarray.Dataset
        Includes rioxarray extension to xarray.Dataset
    """

    da = rioxarray.open_rasterio(geotif_file_path, masked=True)

    # Extract bands and assign as variables in xr.Dataset()
    ds = xr.Dataset()
    for i, v in enumerate(da.band):
        da_tmp = da.sel(band=v)
        da_tmp.name = "band" + str(i + 1)

        ds[da_tmp.name] = da_tmp

    # Delete empty band coordinates.
    # Need to preserve spatial_ref coordinate, even though it appears empty.
    # See spatial_ref attributes under ds.coords.variables used by rioxarray extension.
    del ds.coords["band"]

    # Preserve top-level attributes and extract single value from value iterables e.g. (1,) --> 1
    ds.attrs = da.attrs
    for key, value in ds.attrs.items():
        try:
            if len(value) == 1:
                ds.attrs[key] = value[0]
        except TypeError:
            pass

    return ds

def hyp3_to_xarray(hyp3_dir, epsg, file_type='corr', interpolate_na=False):
    
    dirs = os.listdir(hyp3_dir) #list generated interferograms
    datasets = []
    acquisition_list = []
    
    for idir in dirs:
        tif_path = glob(f'{hyp3_dir}/{idir}/*/*{file_type}.tif')[0]
        tif_fn = os.path.split(tif_path)[-1]
        dates = f'{tif_fn[14:22]}_{tif_fn[23:31]}' #parse filename for interferogram dates
        start_date = datetime.strptime(dates[:8], '%Y%m%d')
        end_date = datetime.strptime(dates[-8:], '%Y%m%d')
        t_baseline = end_date - start_date
        middle_date = t_baseline/2+start_date
        
        src = xr_read_geotif(tif_path, masked=False) #read product to xarray ds
        src = src.assign_coords({"dates": dates})
        src = src.expand_dims("dates")
        
        src = src.assign_coords(start_date = ('dates', [start_date]))
        src = src.assign_coords(end_date = ('dates', [end_date]))
        src = src.assign_coords(t_baseline = ('dates', [t_baseline]))
        src = src.assign_coords(middle_date = ('dates', [middle_date]))
        
        src = src.rename({'band1':file_type})
        src = src.rio.write_crs(epsg)

        # # open metadata file
        # txt_path = glob(f'{hyp3_dir}/{idir}/*/*FFTS.txt')[0]
        # meta_dict = parse_file(txt_path)
        # src = src.assign_coords(perp_baseline = ('dates', [meta_dict['Baseline']]))
        
        if interpolate_na == True:
            src = src.interpolate_na(dim='x', method='linear').fillna(value=0)
        
        datasets.append(src)
        #print(src[file_type].shape, src.dates.item())
       
    ds = xr.concat(datasets, dim="dates", combine_attrs="no_conflicts") #create dataset
    ds = ds.sortby('dates')

    return ds 

In [7]:
asc_corr_ds = hyp3_to_xarray(asc_burst_path, crs, file_type=f'corr_{dam_name}')
des_corr_ds = hyp3_to_xarray(des_burst_path, crs, file_type=f'corr_{dam_name}')

KeyboardInterrupt: 

In [ ]:
# plot mean spatial coherence
padding = 4000

f, ax = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
asc_corr_ds[f'corr_{dam_name}'].mean(dim='dates').plot(ax=ax[0], vmin=0.2, vmax=1, cbar_kwargs= {'label':'coherence'})
dam_gdf.plot(ax=ax[0], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[0], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[0], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[0], edgecolor='white', facecolor='none')
ax[0].set_title('ascending mean coherence')
ax[0].set_aspect('equal')
ax[0].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[0].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[0].set_xlabel('')
ax[0].set_ylabel('')

des_corr_ds[f'corr_{dam_name}'].mean(dim='dates').plot(ax=ax[1], vmin=0.2, vmax=1, cbar_kwargs= {'label':'coherence'})
dam_gdf.plot(ax=ax[1], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[1], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[1], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[1], edgecolor='white', facecolor='none')
ax[1].set_title('descending mean coherence')
ax[1].set_aspect('equal')
ax[1].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[1].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
f.tight_layout()
ax[1].set_xlabel('')
ax[1].set_ylabel('')

plt.savefig(f'./figs/{dam_name}_coherence_map.png', dpi=300)

In [ ]:
# save coherence maps
asc_corr_ds[f'corr_{dam_name}'].mean(dim='dates').rio.to_raster(f'{dam_name}_asc_mean_coherence.tif')
des_corr_ds[f'corr_{dam_name}'].mean(dim='dates').rio.to_raster(f'{dam_name}_des_mean_coherence.tif')
combined_corr = (asc_corr_ds[f'corr_{dam_name}'].mean(dim='dates') + des_corr_ds[f'corr_{dam_name}'].mean(dim='dates'))/2
combined_corr.rio.to_raster(f'{dam_name}_combined_mean_coherence.tif')

In [ ]:
# # clip datasets to dam polygon
# asc_dam_corr_ds = asc_corr_ds.rio.clip(dam_gdf.geometry.values, crs=dam_gdf.crs, drop=True)
# des_dam_corr_ds = des_corr_ds.rio.clip(dam_gdf.geometry.values, crs=dam_gdf.crs, drop=True)

# clip datasets to moving polygon
asc_dam_corr_ds = asc_corr_ds.rio.clip(moving_gdf.geometry.values, crs=moving_gdf.crs, drop=True)
des_dam_corr_ds = des_corr_ds.rio.clip(moving_gdf.geometry.values, crs=moving_gdf.crs, drop=True)

# clip datasets to stable polygon
asc_stable_corr_ds = asc_corr_ds.rio.clip(stable_gdf.geometry.values, crs=stable_gdf.crs, drop=True)
des_stable_corr_ds = des_corr_ds.rio.clip(stable_gdf.geometry.values, crs=stable_gdf.crs, drop=True)

In [ ]:
t_baselines = np.union1d(des_dam_corr_ds.t_baseline.dt.days.values, asc_dam_corr_ds.t_baseline.dt.days.values)

# plot seasonal changes in coherence 
colors = plt.cm.viridis(np.linspace(0, 1, len(t_baselines)))

f, ax = plt.subplots(2, 2, figsize=(7.5, 4), sharex=True, sharey=True, gridspec_kw={'height_ratios': [2.5, 1]})

for i, days in enumerate(reversed(t_baselines)):

    ax[0, 0].scatter(asc_dam_corr_ds.middle_date[asc_dam_corr_ds.t_baseline.dt.days == days].dt.dayofyear,
                  asc_dam_corr_ds[f'corr_{dam_name}'].mean(dim=['x', 'y'])[asc_dam_corr_ds.t_baseline.dt.days == days],
                  color=colors[i],
                  s=7,
                  label=f'{int(days)} days')
    ax[0, 0].tick_params(left=False)

    ax[0, 1].scatter(des_dam_corr_ds.middle_date[des_dam_corr_ds.t_baseline.dt.days == days].dt.dayofyear,
                  des_dam_corr_ds[f'corr_{dam_name}'].mean(dim=['x', 'y'])[des_dam_corr_ds.t_baseline.dt.days == days],
                  color=colors[i],
                  s=7,
                  label=f'{int(days)} days')
    ax[0, 1].tick_params(left=False)

    ax[1, 0].scatter(asc_stable_corr_ds.middle_date[asc_stable_corr_ds.t_baseline.dt.days == days].dt.dayofyear,
                  asc_stable_corr_ds[f'corr_{dam_name}'].mean(dim=['x', 'y'])[asc_stable_corr_ds.t_baseline.dt.days == days],
                  color=colors[i],
                  s=7,
                  label=f'{int(days)} days')

    ax[1, 1].scatter(des_stable_corr_ds.middle_date[des_stable_corr_ds.t_baseline.dt.days == days].dt.dayofyear,
                  des_stable_corr_ds[f'corr_{dam_name}'].mean(dim=['x', 'y'])[des_stable_corr_ds.t_baseline.dt.days == days],
                  color=colors[i],
                  s=7,
                  label=f'{int(days)} days')

    ax[0, 0].set_ylabel('mean coherence')
    ax[1, 0].set_ylabel('mean coherence')
    ax[1, 0].set_xlabel('day of year')
    ax[1, 1].set_xlabel('day of year')
    ax[0, 0].set_title('moving area ascending')
    ax[0, 1].set_title('moving area descending')
    ax[1, 0].set_title('stable area ascending')
    ax[1, 1].set_title('stable area descending')
    #ax[1, 1].legend(loc='lower right', fontsize=10, ncol=2)

    
f.tight_layout()
plt.savefig(f'./figs/{dam_name}_coherence_ts.png', dpi=300)

In [ ]:
# open mask layers
asc_mask = xr.open_dataarray(f'{dam_name}_asc_valid.tif').squeeze()
des_mask = xr.open_dataarray(f'{dam_name}_des_valid.tif').squeeze()
combined_mask = xr.open_dataarray(f'{dam_name}_combined_valid.tif').squeeze()

In [ ]:
# calculate seasonal coherence change
asc_corr_ds_12day = asc_corr_ds.where(asc_corr_ds.t_baseline.dt.days == 12 , drop=True)
des_corr_ds_12day = des_corr_ds.where(des_corr_ds.t_baseline.dt.days == 12 , drop=True)

# low coherence period 
asc_lowcorr_ds = asc_corr_ds_12day.where((asc_corr_ds_12day.start_date.dt.dayofyear > 240) & (asc_corr_ds_12day.end_date.dt.dayofyear < 300), drop=True)
des_lowcorr_ds = des_corr_ds_12day.where((des_corr_ds_12day.start_date.dt.dayofyear > 240) & (des_corr_ds_12day.end_date.dt.dayofyear < 300), drop=True)

# high coherence period
asc_highcorr_ds = asc_corr_ds_12day.where((asc_corr_ds_12day.start_date.dt.dayofyear > 130) & (asc_corr_ds_12day.end_date.dt.dayofyear < 200), drop=True)
des_highcorr_ds = des_corr_ds_12day.where((des_corr_ds_12day.start_date.dt.dayofyear > 130) & (des_corr_ds_12day.end_date.dt.dayofyear < 200), drop=True)

# combined coherence
asc_lowcorr_da = asc_lowcorr_ds.median(dim='dates')[f'corr_{dam_name}']
des_lowcorr_da = des_lowcorr_ds.median(dim='dates')[f'corr_{dam_name}']
lowcorr_da = (asc_lowcorr_da + des_lowcorr_da)/2
lowcorr_da = xr.where(asc_mask == 0, des_lowcorr_da, lowcorr_da)
lowcorr_da = xr.where(des_mask == 0, asc_lowcorr_da, lowcorr_da)
lowcorr_da = xr.where((asc_mask == 0) & (des_mask == 0), np.nan, lowcorr_da)

asc_highcorr_da = asc_highcorr_ds.median(dim='dates')[f'corr_{dam_name}']
des_highcorr_da = des_highcorr_ds.median(dim='dates')[f'corr_{dam_name}']
highcorr_da = (asc_highcorr_da + des_highcorr_da)/2
highcorr_da = xr.where(asc_mask == 0, des_highcorr_da, highcorr_da)
highcorr_da = xr.where(des_mask == 0, asc_highcorr_da, highcorr_da)
highcorr_da = xr.where((asc_mask == 0) & (des_mask == 0), np.nan, highcorr_da)

# save seasonal coherence change
corr_change_da = (highcorr_da - lowcorr_da)
corr_change_da.rio.to_raster(f'{dam_name}_seasonal_coherence_change.tif')
ice_da = xr.where(corr_change_da >= 0.03, 1, 0).astype('float32')
ice_da.rio.to_raster(f'{dam_name}_ice_map.tif')

## stack for velocity

In [ ]:
# open interferograms
asc_unw_ds = hyp3_to_xarray(asc_burst_path, crs, file_type=f'unw_phase_{dam_name}')
des_unw_ds = hyp3_to_xarray(des_burst_path, crs, file_type=f'unw_phase_{dam_name}')

asc_corr_ds['unw_phase'] = asc_unw_ds[f'unw_phase_{dam_name}']
des_corr_ds['unw_phase'] = des_unw_ds[f'unw_phase_{dam_name}']

In [ ]:
# remove layover and radar shadow
asc_corr_ds['unw_phase'] = asc_corr_ds['unw_phase'].where(asc_mask == 1)
des_corr_ds['unw_phase'] = des_corr_ds['unw_phase'].where(des_mask == 1)

# limit to short baselines
asc_ds_12day = asc_corr_ds.where(asc_corr_ds.t_baseline.dt.days <= 12, drop=True)
des_ds_12day = des_corr_ds.where(des_corr_ds.t_baseline.dt.days <= 12, drop=True)

# limit to warm period 
asc_warm_ds = asc_ds_12day.where((asc_ds_12day.start_date.dt.dayofyear > 200) & (asc_ds_12day.end_date.dt.dayofyear < 300), drop=True)
des_warm_ds = des_ds_12day.where((des_ds_12day.start_date.dt.dayofyear > 200) & (des_ds_12day.end_date.dt.dayofyear < 300), drop=True)

# filter pixel by pixel
asc_highcorr_ds = asc_warm_ds.where(asc_warm_ds[f'corr_{dam_name}'] > 0.6)
des_highcorr_ds = des_warm_ds.where(des_warm_ds[f'corr_{dam_name}'] > 0.6)

# valid pixel count maps
asc_count = asc_highcorr_ds.unw_phase.count(dim='dates')
des_count = des_highcorr_ds.unw_phase.count(dim='dates')

# convert to m
asc_highcorr_ds['displacement'] = asc_highcorr_ds['unw_phase']*0.05546576/12.5663706
des_highcorr_ds['displacement'] = des_highcorr_ds['unw_phase']*0.05546576/12.5663706

# calculate velocity
asc_highcorr_ds['velocity'] = (asc_highcorr_ds['displacement']/asc_highcorr_ds.t_baseline.dt.days)*365.25
des_highcorr_ds['velocity'] = (des_highcorr_ds['displacement']/des_highcorr_ds.t_baseline.dt.days)*365.25

# remove reference area displacement
asc_highcorr_ds['velocity'] = asc_highcorr_ds['velocity'] - asc_highcorr_ds['velocity'].rio.clip(reference_gdf.geometry, reference_gdf.crs).median(dim=['x', 'y'])
des_highcorr_ds['velocity'] = des_highcorr_ds['velocity'] - des_highcorr_ds['velocity'].rio.clip(reference_gdf.geometry, reference_gdf.crs).median(dim=['x', 'y'])

asc_velocity = asc_highcorr_ds['velocity'].median(dim='dates')
des_velocity = des_highcorr_ds['velocity'].median(dim='dates')

# calculate nmad 
asc_velocity_nmad = (np.fabs(asc_highcorr_ds.velocity - asc_velocity)).median(dim='dates') * 1.4826
des_velocity_nmad = (np.fabs(des_highcorr_ds.velocity - des_velocity)).median(dim='dates') * 1.4826

# remove low-count pixels
asc_velocity = asc_velocity.where(asc_count >= 5)
des_velocity = des_velocity.where(des_count >= 5)

In [ ]:
# plot mean spatial coherence
padding = 4000

f, ax = plt.subplots(3, 2, figsize=(12, 15), sharex=True, sharey=True)
asc_velocity.plot(ax=ax[0, 0], vmin=-0.2, vmax=0.2, cmap='RdBu_r', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[0, 0], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[0, 0], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[0, 0], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[0, 0], edgecolor='white', facecolor='none')
ax[0, 0].set_title('ascending median velocity')
ax[0, 0].set_aspect('equal')
ax[0, 0].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[0, 0].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[0, 0].set_xlabel('')
ax[0, 0].set_ylabel('')

des_velocity.plot(ax=ax[0, 1], vmin=-0.2, vmax=0.2, cmap='RdBu_r', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[0, 1], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[0, 1], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[0, 1], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[0, 1], edgecolor='white', facecolor='none')
ax[0, 1].set_title('descending median velocity')
ax[0, 1].set_aspect('equal')
ax[0, 1].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[0, 1].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)

asc_velocity_nmad.plot(ax=ax[1, 0], vmin=0, vmax=0.5, cmap='magma', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[1, 0], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[1, 0], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[1, 0], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[1, 0], edgecolor='white', facecolor='none')
ax[1, 0].set_title('ascending velocity nmad')
ax[1, 0].set_aspect('equal')
ax[1, 0].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[1, 0].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[1, 0].set_xlabel('')
ax[1, 0].set_ylabel('')

des_velocity_nmad.plot(ax=ax[1, 1], vmin=0, vmax=0.5, cmap='magma', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[1, 1], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[1, 1], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[1, 1], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[1, 1], edgecolor='white', facecolor='none')
ax[1, 1].set_title('descending velocity nmad')
ax[1, 1].set_aspect('equal')
ax[1, 1].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[1, 1].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[1, 1].set_xlabel('')
ax[1, 1].set_ylabel('')

asc_count.plot(ax=ax[2, 0], cmap='plasma', cbar_kwargs= {'label':'pixel count'})
dam_gdf.plot(ax=ax[2, 0], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[2, 0], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[2, 0], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[2, 0], edgecolor='white', facecolor='none')
ax[2, 0].set_title('ascending pixel count')
ax[2, 0].set_aspect('equal')
ax[2, 0].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[2, 0].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[2, 0].set_xlabel('')
ax[2, 0].set_ylabel('')

des_count.plot(ax=ax[2, 1], cmap='plasma', cbar_kwargs= {'label':'pixel count'})
dam_gdf.plot(ax=ax[2, 1], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[2, 1], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[2, 1], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[2, 1], edgecolor='white', facecolor='none')
ax[2, 1].set_title('descending pixel count')
ax[2, 1].set_aspect('equal')
ax[2, 1].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[2, 1].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[2, 1].set_xlabel('')
ax[2, 1].set_ylabel('')

f.tight_layout()
plt.savefig(f'./figs/{dam_name}_los_veloc_map.png', dpi=300)

In [ ]:
# save to rasters
asc_velocity.rio.to_raster(f'{dam_name}_asc_median_velocity.tif')
asc_velocity_nmad.rio.to_raster(f'{dam_name}_asc_velocity_nmad.tif')
des_velocity.rio.to_raster(f'{dam_name}_des_median_velocity.tif')
des_velocity_nmad.rio.to_raster(f'{dam_name}_des_velocity_nmad.tif')

## ew ud

In [ ]:
# open look angle files
asc_lv_phi_ds = hyp3_to_xarray(asc_burst_path, crs, file_type=f'lv_phi_{dam_name}')
des_lv_phi_ds = hyp3_to_xarray(des_burst_path, crs, file_type=f'lv_phi_{dam_name}')
asc_lv_theta_ds = hyp3_to_xarray(asc_burst_path, crs, file_type=f'lv_theta_{dam_name}')
des_lv_theta_ds = hyp3_to_xarray(des_burst_path, crs, file_type=f'lv_theta_{dam_name}')

In [ ]:
asc_azimuth = (90 - (270 - ((asc_lv_phi_ds[f'lv_phi_{dam_name}'].mean(dim='dates')*270)/-np.pi)))
des_azimuth = ((des_lv_phi_ds[f'lv_phi_{dam_name}'].mean(dim='dates')*270)/-np.pi)+270
asc_incidence = 90 - ((asc_lv_theta_ds[f'lv_theta_{dam_name}'].mean(dim='dates')*90)/(np.pi/2))
des_incidence = 90 - ((des_lv_theta_ds[f'lv_theta_{dam_name}'].mean(dim='dates')*90)/(np.pi/2))

# construct unit look vectors 
asc_n_hat = np.cos(np.radians(asc_azimuth))*np.sin(np.radians(asc_incidence))
asc_e_hat = np.sin(np.radians(asc_azimuth))*np.sin(np.radians(asc_incidence))
asc_z_hat = -np.cos(np.radians(asc_incidence))

des_n_hat = np.cos(np.radians(des_azimuth))*np.sin(np.radians(des_incidence))
des_e_hat = np.sin(np.radians(des_azimuth))*np.sin(np.radians(des_incidence))
des_z_hat = -np.cos(np.radians(des_incidence))

# construct ud and ew unit vectors [n, e, d]
ud_unit_vector = np.array([np.zeros_like(des_n_hat.values), np.zeros_like(des_n_hat.values), np.ones_like(des_n_hat.values)])
ew_unit_vector = np.array([np.zeros_like(des_n_hat.values), np.ones_like(des_n_hat.values), np.zeros_like(des_n_hat.values)])

# find dot products for LOS
asc_ud = np.einsum('ijk,ijk->jk', np.array([asc_n_hat.values, asc_e_hat.values, asc_z_hat.values]), ud_unit_vector)
asc_ew = np.einsum('ijk,ijk->jk', np.array([asc_n_hat.values, asc_e_hat.values, asc_z_hat.values]), ew_unit_vector)
des_ud = np.einsum('ijk,ijk->jk', np.array([des_n_hat.values, des_e_hat.values, des_z_hat.values]), ud_unit_vector)
des_ew = np.einsum('ijk,ijk->jk', np.array([des_n_hat.values, des_e_hat.values, des_z_hat.values]), ew_unit_vector)

# solve for horizontal and vertical displacement for all dates 
ud_velocity = (asc_ew*des_velocity - des_ew*asc_velocity)/(asc_ew*des_ud - des_ew*asc_ud)
ew_velocity = ((asc_ud*ud_velocity) - asc_velocity)/-asc_ew

In [ ]:
# plot mean spatial coherence
padding = 4000

f, ax = plt.subplots(1, 2, figsize=(12, 5), sharex=True, sharey=True)
ud_velocity.plot(ax=ax[0], vmin=-0.2, vmax=0.2, cmap='RdBu', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[0], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[0], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[0], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[0], edgecolor='white', facecolor='none')
ax[0].set_title('ud median velocity')
ax[0].set_aspect('equal')
ax[0].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[0].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)
ax[0].set_xlabel('')
ax[0].set_ylabel('')

ew_velocity.plot(ax=ax[1], vmin=-0.5, vmax=0.5, cmap='BrBG', cbar_kwargs= {'label':'velocity (m/yr)'})
dam_gdf.plot(ax=ax[1], edgecolor='black', facecolor='none')
moving_gdf.plot(ax=ax[1], edgecolor='red', facecolor='none')
stable_gdf.plot(ax=ax[1], edgecolor='blue', facecolor='none')
reference_gdf.plot(ax=ax[1], edgecolor='white', facecolor='none')
ax[1].set_title('ew median velocity')
ax[1].set_aspect('equal')
ax[1].set_xlim(aoi_extent[0]+padding, aoi_extent[2]-padding)
ax[1].set_ylim(aoi_extent[3]+padding, aoi_extent[1]-padding)

f.tight_layout()

plt.savefig(f'./figs/{dam_name}_decomposed_veloc_map.png', dpi=300)

In [ ]:
# save to rasters
ud_velocity.rio.to_raster(f'{dam_name}_ud_median_velocity.tif')
ew_velocity.rio.to_raster(f'{dam_name}_ew_median_velocity.tif')

In [ ]:
# combined velocity
np.sqrt(ud_velocity**2 + ew_velocity**2).rio.to_raster(f'{dam_name}_combined_median_velocity.tif')